 Import Libraries

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import matplotlib.pyplot as plt
import seaborn as sns


Load CSV File

In [ ]:
df = pd.read_csv("decision_regret_dataset.csv")

print(df.head())
print(df.info())


Clean & Prepare Target Column

In [ ]:
# Convert Yes/No to 1/0
df['1️⃣3️⃣ Do you regret this decision now?'] = df['1️⃣3️⃣ Do you regret this decision now?'].map({
    "Yes": 1,
    "No": 0
})

# Rename to simple name
df.rename(columns={
    '1️⃣3️⃣ Do you regret this decision now?': 'Regret'
}, inplace=True)


Remove Unnecessary Columns

In [ ]:
df.drop(columns=['Timestamp'], inplace=True)


Encode All Categorical Columns

In [ ]:
df = pd.get_dummies(df, drop_first=True)


Split Features and Target

In [ ]:
X = df.drop("Regret", axis=1)
y = df["Regret"]


Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)


Train Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=100,
    random_state=42
)

model.fit(X_train, y_train)


 Make Predictions

In [ ]:
y_pred = model.predict(X_test)


Check Model Performance

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred)

sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()


Classification Report

In [ ]:
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': model.feature_importances_
}).sort_values(by='Importance', ascending=False)




from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))


In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(x='Importance', y='Feature', data=feature_importance.head(10))
plt.title("Top 10 Important Features")
plt.show()


In [ ]:
def get_valid_input(prompt, min_val, max_val):
    while True:
        try:
            value = int(input(prompt))
            if value < min_val or value > max_val:
                print(f"❌ Please enter a value between {min_val} and {max_val}")
            else:
                return value
        except ValueError:
            print("❌ Please enter a valid integer number.")


# Taking validated input
age = get_valid_input("Enter Age (15-80): ", 15, 80)
confidence = get_valid_input("Confidence level (1-10): ", 1, 10)
emotion = get_valid_input("Emotional stability (1-10): ", 1, 10)
pressure = get_valid_input("External pressure (1-5): ", 1, 5)
satisfaction = get_valid_input("Satisfaction level (1-10): ", 1, 10)


new_person = {
    '1️⃣ Age': age,
    '7️⃣ How confident were you when making this decision?': confidence,
    '8️⃣ How emotionally stable were you at that time?': emotion,
    '9️⃣ How much external pressure did you feel (family, society, friends)?': pressure,
    '1️⃣2️⃣ How satisfied are you with the outcome of this decision?': satisfaction
}

new_df = pd.DataFrame([new_person])
new_df = pd.get_dummies(new_df)
new_df = new_df.reindex(columns=X.columns, fill_value=0)

prediction = model.predict(new_df)
probability = model.predict_proba(new_df)[0][1]

if prediction[0] == 1:
    print("\n⚠️ Model says: You may regret this decision.")
else:
    print("\n✅ Model says: You are unlikely to regret this decision.")

print(f"Regret Probability: {probability*100:.2f}%")


KeyboardInterrupt: Interrupted by user

Personal decision regert predictor

In [ ]:
import joblib

joblib.dump(model, 'my_trained_model.pkl')

print("✅ Model Saved Successfully!")